In [1]:
from astropy.coordinates import SkyCoord, get_moon, EarthLocation, ICRS, GCRS,AltAz
from astropy.time import Time
from astropy.io.fits.hdu.hdulist import HDUList
from datetime import date
from astral import Astral, Location
from astropy.time import Time
import astropy.units as u

In [10]:
import pyfits
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter
from scipy.interpolate import interp1d

In [2]:
## Finding out which file years to download
all_sky=pd.read_csv('objSKY.csv')
all_sky.head()
print('earliest date=', min(all_sky['MJD']), 'or 12-11-2009')
print('latest date=', max(all_sky['MJD']), 'or 05-12-2016')

earliest date= 55176 or 12-11-2009
latest date= 57520 or 05-12-2016


## Data

In [3]:
## To check for reading in the right file
rand_files=('spec-3586-55181-0496.fits','spec-3586-55181-0788.fits','spec-3586-55181-0996.fits',
            'spec-3761-55272-0008.fits','spec-10000-57346-0334.fits','spec-3761-55272-0475.fits',
            'spec-10000-57346-0659.fits','spec-5478-56014-0654.fits','spec-10000-57346-0955.fits',
            'spec-5478-56014-0716.fits','spec-10000-57346-0994.fits')

## To look for patterns
inseq_files=('spec-3663-55176-0010.fits','spec-3663-55176-0012.fits','spec-3663-55176-0020.fits',
             'spec-3663-55176-0024.fits','spec-3663-55176-0036.fits','spec-3663-55176-0038.fits',
             'spec-3663-55176-0048.fits','spec-3663-55176-0052.fits','spec-3663-55176-0056.fits',
             'spec-3663-55176-0068.fits','spec-3663-55176-0075.fits','spec-3663-55176-0078.fits',
             'spec-3663-55176-0090.fits','spec-3663-55176-0094.fits','spec-3663-55176-0096.fits',
             'spec-3663-55176-0108.fits','spec-3663-55176-0112.fits','spec-3663-55176-0114.fits',
             'spec-3663-55176-0128.fits','spec-3663-55176-0134.fits')

solar_files=('g2009.txt','g2010.txt','g2011.txt','g2012.txt','g2013.txt','g2014.txt','g2015.txt','g2016.txt')

## Getting MJDs from fits

In [4]:
#a) first load all files and convert all dates into mjds.
def get_mjd(file):
    da=pyfits.open(file)
    return (da[4].header['MJD'])

In [5]:
for i in rand_files:
    print(get_mjd(i))

55181
55181
55181
55272
57345
55272
57345
56014
57345
56014
57345


## Organizing by date and sum duplicates

In [6]:
def merge_subs(lst_of_lsts):
    res = []
    for row in lst_of_lsts:
        for i, resrow in enumerate(res):
            if row[:3]==resrow[:3]:
                res[i] += row[1:]
                break
        else:
            res.append(row)
    return (res)
## Merges lists, but does not remove first 3 cols

In [7]:
def f2(seq): 
    # order preserving
    checked = []
    for e in seq:
        if e not in checked:
            checked.append(e)
    return checked
## removes duplicates

In [8]:
## Function to organize mill. of sol. disk by date
def organize_sum_dates(txt_file):
    ## We only need year, month, day of month, and mill. of sol. disk
    ## Cols         1-4    5-6      7-8               31-34
    demo=list(chain.from_iterable((x[:4], x[4:6], x[6:10], x[31:34]) for x in open(txt_file).readlines()))
    ## Organizing cols
    str_list=[demo[i*4:i*4+4] for i in range(int(len(demo)/4))]
    ## Converting to float
    float_dates=[]
    for k in range(len(str_list)):
        float_dates.append([float(i) for i in str_list[k]])
    b=([x[:4] for x in float_dates])
    ve=merge_subs(b)
    new=[]
    summed_mills=[]
    no_mills=[]
    merged_lists=[]
    for i in range(len(ve)):
        new.append(f2(ve[i]))
        summed_mills.append(sum(new[i][3:]))
        no_mills.append(new[i][:3])
        merged_lists.append(np.append(no_mills[i],summed_mills[i]).tolist())
    return(merged_lists)

In [9]:
trial=organize_sum_dates(solar_files[2])
trial[:5]

[[2011.0, 1.0, 1.5, 405.0],
 [2011.0, 1.0, 2.5, 375.0],
 [2011.0, 1.0, 3.5, 517.0],
 [2011.0, 1.0, 4.5, 537.0],
 [2011.0, 1.0, 5.5, 381.0]]

In [8]:
#for i in solar_files:
#    print(organize_by_date(i))

## Interpolated function from MJDs to mill of sol. disk

In [65]:
#b) Then set up one massive interpolated function from mjds to mill of sol. disk
## Pulling out cols of interest
demo=list(chain.from_iterable((x[:4], x[4:6], x[6:10], x[31:34]) for x in open('g2010.txt').readlines()))
## Organizing cols
str_list=[demo[i*4:i*4+4] for i in range(int(len(demo)/4))]
## Converting to float
float_dates=[]
for k in range(len(str_list)):
    float_dates.append([float(i) for i in str_list[k]])
b=([x[:4] for x in float_dates])
ve=merge_subs(b)
new=[]
summed_mills=[]
no_mills=[]
merged_lists=[]
for i in range(len(ve)):
    new.append(f2(ve[i]))
    summed_mills.append(sum(new[i][3:]))
    no_mills.append(new[i][:3])
    merged_lists.append(np.append(no_mills[i],summed_mills[i]).tolist())

In [66]:
merged_lists[1][2]+1

3.5

In [72]:
for i in range(len(merged_lists)):
    if merged_lists[i][2]!=merged_lists[i-1][2]+1 and merged_lists[i][1]==merged_lists[i-1][1]:#missing days in months
        print(merged_lists[i-1], merged_lists[i])
        print(np.interp(2.5,merged_lists[i-1], merged_lists[i]))
        

[2010.0, 1.0, 5.5, 0.0] [2010.0, 1.0, 7.5, 67.0]
67.0
[2010.0, 1.0, 18.5, 21.0] [2010.0, 1.0, 20.5, 28.0]
2010.0
[2010.0, 3.0, 5.5, 11.0] [2010.0, 3.0, 10.5, 10.0]
2010.0
[2010.0, 4.0, 10.5, 1.0] [2010.0, 4.0, 12.5, 19.0]
19.0
[2010.0, 4.0, 14.5, 16.0] [2010.0, 4.0, 28.5, 18.0]
2010.0
[2010.0, 4.0, 28.5, 18.0] [2010.0, 4.0, 30.5, 33.0]
2010.0
[2010.0, 5.0, 8.5, 7.0] [2010.0, 5.0, 21.5, 56.0]
2010.0
[2010.0, 6.0, 14.5, 1.0] [2010.0, 6.0, 17.5, 15.0]
15.0
[2010.0, 6.0, 25.5, 0.0] [2010.0, 6.0, 27.5, 84.0]
84.0
[2010.0, 8.0, 20.5, 0.0] [2010.0, 8.0, 24.5, 12.0]
12.0
[2010.0, 9.0, 8.5, 0.0] [2010.0, 9.0, 11.5, 37.0]
37.0
[2010.0, 10.0, 5.5, 0.0] [2010.0, 10.0, 8.5, 0.0]
0.0
[2010.0, 12.0, 17.5, 0.0] [2010.0, 12.0, 25.5, 44.0]
44.0


In [16]:
#list_example=[[1,1,2016,212], [1,2,2016,170], [1,3,2016,150], [1,5,2016,96], [1,6,2016,150], [1,8,2016,321]]

In [15]:
#zip(list_example)